# Explore here

In [39]:
# Your code here
import pandas as pd
import numpy as np
import regex as re
archivo = 'https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv'
total_data  = pd.read_csv(archivo)
total_data.head()


,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [40]:
print(total_data.columns)
total_data.head() 

Index(['url', 'is_spam'], dtype='object')


,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [41]:
print(total_data.head(10))

                                                 url  is_spam
0  https://briefingday.us8.list-manage.com/unsubs...     True
1                             https://www.hvper.com/     True
2                 https://briefingday.com/m/v4n3i4f3     True
3   https://briefingday.com/n/20200618/m#commentform    False
4                        https://briefingday.com/fan     True
5  https://www.brookings.edu/interactives/reopeni...    False
6  https://www.reuters.com/investigates/special-r...    False
7  https://www.theatlantic.com/magazine/archive/2...    False
8  https://www.vox.com/2020/6/17/21294680/john-bo...    False
9  https://www.theguardian.com/travel/2020/jun/18...    False


In [42]:
total_data["is_spam_int"] = total_data["is_spam"].astype(int)
print(total_data.head())

                                                 url  is_spam  is_spam_int
0  https://briefingday.us8.list-manage.com/unsubs...     True            1
1                             https://www.hvper.com/     True            1
2                 https://briefingday.com/m/v4n3i4f3     True            1
3   https://briefingday.com/n/20200618/m#commentform    False            0
4                        https://briefingday.com/fan     True            1


transformar en 1 y 0 

In [43]:

total_data.drop(columns=['is_spam'], inplace=True)


print(total_data.head())

                                                 url  is_spam_int
0  https://briefingday.us8.list-manage.com/unsubs...            1
1                             https://www.hvper.com/            1
2                 https://briefingday.com/m/v4n3i4f3            1
3   https://briefingday.com/n/20200618/m#commentform            0
4                        https://briefingday.com/fan            1


eliminar los duplicados 

In [44]:

print(f"Forma del DataFrame antes de eliminar duplicados: {total_data.shape}")

total_data.drop_duplicates(inplace=True)

total_data.reset_index(inplace=True, drop=True)
print(f"Forma del DataFrame después de eliminar duplicados: {total_data.shape}")

print(total_data.head())

Forma del DataFrame antes de eliminar duplicados: (2999, 2)
Forma del DataFrame después de eliminar duplicados: (2369, 2)
                                                 url  is_spam_int
0  https://briefingday.us8.list-manage.com/unsubs...            1
1                             https://www.hvper.com/            1
2                 https://briefingday.com/m/v4n3i4f3            1
3   https://briefingday.com/n/20200618/m#commentform            0
4                        https://briefingday.com/fan            1


Procesamiento del texto

In [45]:
def preprocess_text(text):
    
    text = re.sub(r'[^a-z ]', " ", text)

   
    text = re.sub(r'\s+[a-zA-Z]\s+', " ", text)
    text = re.sub(r'\^[a-zA-Z]\s+', " ", text)

    
    text = re.sub(r'\s+', " ", text.lower())

   
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)

    return text.split()

In [46]:
total_data["url"] = total_data["url"].apply(preprocess_text)
total_data.head()

,url,is_spam_int
0,"[https, briefingday, us, list, manage, com, un...",1
1,"[https, www, hvper, com]",1
2,"[https, briefingday, com, v, i]",1
3,"[https, briefingday, com, m, commentform]",0
4,"[https, briefingday, com, fan]",1


Lematizacion 

In [49]:
def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmatized_tokens)

total_data["url_lemmatized"] = total_data["url"].apply(lemmatize_tokens)
print(total_data[['url', 'url_lemmatized']].head())

                                                 url  \
0  [https, briefingday, us, list, manage, com, un...   
1                           [https, www, hvper, com]   
2                    [https, briefingday, com, v, i]   
3          [https, briefingday, com, m, commentform]   
4                     [https, briefingday, com, fan]   

                                   url_lemmatized  
0  http briefingday u list manage com unsubscribe  
1                              http www hvper com  
2                        http briefingday com v i  
3              http briefingday com m commentform  
4                        http briefingday com fan  


NameError: name 'lemmatize_text' is not defined